# **Distribución % de créditos por organismo**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "queretaro" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_queretaro',
 'df_financiamientos_2020_queretaro',
 'df_financiamientos_2021_queretaro',
 'df_financiamientos_2022_queretaro',
 'df_financiamientos_2023_queretaro',
 'df_financiamientos_2024_queretaro']

## 2022

In [2]:
finance = df_financiamientos_2022_queretaro[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
8868,8869,3
8869,8870,5
8870,8871,7
8871,8872,26


In [3]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']


# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO      18
1        CNBV    4895
2      CONAVI       8
3    FOVISSTE     521
4   INFONAVIT    3398
5       INSUS      27


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1994808463.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '7' '26' '26']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  finance.loc[:, 'organismo'] = finance['organismo'].astype(str)


In [4]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,4895,55.20
1,INFONAVIT,3398,38.32
2,FOVISSTE,521,5.88
3,INSUS,27,0.30
4,BANJERCITO,18,0.20
5,CONAVI,8,0.09


In [5]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  17734  registros en la base
Hay  8867  registros al contar


In [6]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
1,INFONAVIT,3398,38.32
0,CNBV,4895,55.20
2,FOVISSTE,521,5.88
4,BANJERCITO,18,0.20
3,INSUS,27,0.30
5,CONAVI,8,0.09


In [7]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
<>:31: SyntaxWarning: invalid escape sequence '\g'
<>:47: SyntaxWarning: invalid escape sequence '\g'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1935841576.py:31: SyntaxWarning: invalid escape sequence '\g'
  def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1935841576.py:47: SyntaxWarning: invalid escape sequence '\g'
  guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2022', carpeta='assets\graficas')


In [8]:
38.32+55.20+5.88+0.20+0.30+0.09

99.99000000000001

## 2023

In [9]:
finance = df_financiamientos_2023_queretaro[['id','organismo']]
finance

,id,organismo
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
8359,8360,3
8360,8361,3
8361,8362,3
8362,8363,3


In [10]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']


# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO      13
1        CNBV    4144
2      CONAVI       4
3    FOVISSTE     391
4   INFONAVIT    3796
5       INSUS      11


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\4180320801.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '3' '3' '3']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [11]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,CNBV,4144,49.58
1,INFONAVIT,3796,45.41
2,FOVISSTE,391,4.68
3,BANJERCITO,13,0.16
4,INSUS,11,0.13
5,CONAVI,4,0.05


In [12]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  16718  registros en la base
Hay  8359  registros al contar


In [13]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
1,INFONAVIT,3796,45.41
0,CNBV,4144,49.58
2,FOVISSTE,391,4.68
3,BANJERCITO,13,0.16
4,INSUS,11,0.13
5,CONAVI,4,0.05


In [14]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,    
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2023', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1871607032.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1871607032.py:47: SyntaxWarning:

invalid escape sequence '\g'



In [16]:
45.41+49.58+4.68+0.16+0.13+0.05

100.00999999999998

## 2024

In [15]:
finance = df_financiamientos_2024_queretaro[['id','organismo']]
finance

,id,organismo
0,1,4
1,2,4
2,3,4
3,4,4
4,5,4
...,...,...
5430,5431,2
5431,5432,2
5432,5433,2
5433,5434,2


In [16]:
# Convertir la columna 'organismo' a cadena de texto (str)
#finance['organismo'] = finance['organismo'].astype(str)
finance.loc[:, 'organismo'] = finance['organismo'].astype(str)

# Crear un diccionario con las correspondencias entre número y categoría
categorias = {
    '1': 'INFONAVIT',
    '2': 'CNBV',
    '3': 'FOVISSTE',
    '5': 'CONAVI',
    '7': 'BANJERCITO',
    '26': 'INSUS'
}
# Reemplazar los valores de 'organismo' con el texto correspondiente utilizando replace
#finance['organismo'] = finance['organismo'].replace(categorias)
finance.loc[:, 'organismo'] = finance['organismo'].replace(categorias)

# Reemplazar cualquier otro número que no esté en el diccionario con '-'
#finance['organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)
finance.loc[:, 'organismo'] = finance['organismo'].replace(to_replace=r'^\d+$', value='-', regex=True)

# Eliminar la fila donde rango_ingresos es '0'
finance = finance[finance['organismo'] != '-']



# Obtener el conteo de registros por cada categoría en la columna 'organismo'
conteo_organismos = finance['organismo'].value_counts().reset_index()
conteo_organismos.columns = ['organismo', 'conteo']

# Asegurarse de que todas las categorías estén presentes en el conteo, incluso si tienen un conteo de 0
for categoria in categorias.values():
    if categoria not in conteo_organismos['organismo'].unique():
        nuevo_registro = pd.DataFrame({'organismo': [categoria], 'conteo': [0]})
        conteo_organismos = pd.concat([conteo_organismos, nuevo_registro], ignore_index=True)

# Ordenar el DataFrame por las categorías en orden ascendente
conteo_organismos = conteo_organismos.sort_values(by='organismo').reset_index(drop=True)

print(conteo_organismos)

    organismo  conteo
0  BANJERCITO       4
1        CNBV    2270
2      CONAVI      48
3    FOVISSTE     272
4   INFONAVIT    2781
5       INSUS       0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\1800649255.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4' '4' '4' ... '2' '2' '3']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [17]:
# Crear un DataFrame con el conteo de registros por cada organismo
df_conteo_organismos = finance['organismo'].value_counts().reset_index()
df_conteo_organismos.columns = ['organismo', 'conteo']

# Calcular el total de registros en el DataFrame original
total_registros = len(finance)

# Calcular el promedio que ocupa cada organismo con respecto al total de registros
df_conteo_organismos['promedio'] = round((df_conteo_organismos['conteo'] * 100) / total_registros,2)
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,2781,51.74
1,CNBV,2270,42.23
2,FOVISSTE,272,5.06
3,CONAVI,48,0.89
4,BANJERCITO,4,0.07


In [18]:
f_2022 = finance.size
cont = df_conteo_organismos['conteo'].sum()
print("Hay ", f_2022, " registros en la base")
print("Hay ", cont, " registros al contar")

Hay  10750  registros en la base
Hay  5375  registros al contar


In [19]:
# Definir el diccionario de orden de organismos
orden_organismos = {
    'INFONAVIT': 1,
    'CNBV': 2,
    'FOVISSTE': 3,
    'BANJERCITO': 4
}

#df_conteo_organismos['orden'] = df_conteo_organismos['organismo'].map(orden_organismos)
df_conteo_organismos.loc[:, 'orden'] = df_conteo_organismos['organismo'].map(orden_organismos)

# Ordenar el DataFrame por el orden de los organismos
df_conteo_organismos = df_conteo_organismos.sort_values(by='orden')
df_conteo_organismos = df_conteo_organismos.drop(columns=['orden'])
df_conteo_organismos

,organismo,conteo,promedio
0,INFONAVIT,2781,51.74
1,CNBV,2270,42.23
2,FOVISSTE,272,5.06
4,BANJERCITO,4,0.07
3,CONAVI,48,0.89


In [20]:

labels = df_conteo_organismos['organismo']
values = df_conteo_organismos['promedio']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.2f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Porcentaje de créditos por organismo (2022)',
    #title_font_size = 22,
    xaxis=dict(title="Organismo"),
    yaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_creditos_organismo_2024', carpeta='assets\graficas')
fig.show()

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

<>:31: SyntaxWarning:

invalid escape sequence '\g'

<>:47: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\2492648266.py:31: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7376\2492648266.py:47: SyntaxWarning:

invalid escape sequence '\g'



In [23]:
51.74+42.23+5.06+0.07+0.89

99.99